# Feature performance comparison

The purpose of this notebook is to compare the classification performance of the individual features, and their combination, for A. Thaliana

# Imports

In [1]:
import os
import sys
from IPython.display import display

sys.path.append('../src')
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC, LinearSVC
from sklearn.feature_selection import SelectKBest
from sklearn.decomposition import PCA, KernelPCA
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.feature_selection import SelectKBest, RFE, VarianceThreshold
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.base import clone
from scipy.stats import shapiro
import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator,TransformerMixin

from yellowbrick.features import ParallelCoordinates
from yellowbrick.features import Rank1D, Rank2D

import pandas as pd
import numpy as np
import seaborn as sns

from dataset.transporter_dataset import create_dataset
from dataset.cluster_fasta import cd_hit
from features.labels import fasta_to_labels
from features.compositions import calculate_composition_feature
from features.pssm import calculate_pssm_feature
from features.coexp import calculate_coexp_feature
from models.eval import nested_crossval
from visualization.feature_plots import create_plot

# Globals

In [2]:
N_THREADS = 16
IDENTITY_THRESHOLD=70
TAX_ID = 559292
ORGANISM = "yeast"
LOG_FILE = f"../logs/{ORGANISM}_amino_sugar.log"

# Dataset

In [3]:
# Delete previous log
if os.path.exists(LOG_FILE):
    with open(LOG_FILE, "w"):
        pass

create_dataset(
    keywords_substrate_filter=["Amino-acid transport", "Sugar transport"],
    keywords_component_filter=["Membrane"],
    keywords_transport_filter=["Transport"],
    input_file="../data/raw/swissprot/uniprot-reviewed_yes.tab.gz",
    multi_substrate="integrate",
    verbose=True,
    tax_ids_filter=[TAX_ID],
    output_tsv=f"../data/datasets/{ORGANISM}_amino_sugar.tsv",
    output_fasta=f"../data/datasets/{ORGANISM}_amino_sugar.fasta",
    output_log=LOG_FILE,
)



,keywords_transport,keywords_location,keywords_transport_related,gene_names,protein_names,tcdb_id,organism_id,sequence
Uniprot,,,,,,,,
P0CD99,Sugar transport,Cell membrane;Membrane;Transmembrane,Transport,MPH2 YDL247W,Alpha-glucosides permease MPH2 (Maltose transp...,NaN,559292,MKNLSFLINRRKENTSDSNVYPGKAKSHEPSWIEMDDQTKKDGLDI...
Q08986,Amino-acid transport,Endoplasmic reticulum;Membrane;Transmembrane,Transport,SAM3 YPL274W,S-adenosylmethionine permease SAM3 (S-adenosyl...,2.A.3.10.15,559292,MDILKRGNESDKFTKIETESTTIPNDSDRSGSLIRRMKDSFKQSNL...
P38967,Amino-acid transport,Membrane;Transmembrane,Transport,TAT2 LTG3 SAB2 SCM2 TAP2 YOL020W,Tryptophan permease (Tryptophan amino acid tra...,2.A.3.10.8,559292,MTEDFISSVKRSNEELKERKSNFGFVEYKSKQLTSSSSHNSNSSHH...
P39003,Sugar transport,Membrane;Transmembrane,Transport,HXT6 YDR343C D9651.12,High-affinity hexose transporter HXT6,NaN,559292,MSQDAAIAEQTPVEHLSAVDSASHSVLSTPSNKAERDEIKAYGEGE...
P38085,Amino-acid transport,Membrane;Transmembrane,Transport,TAT1 TAP1 VAP1 YBR069C YBR0710,Valine/tyrosine/tryptophan amino-acid permease...,2.A.3.10.9,559292,MDDSVSFIAKEASPAQYSHSLHERTHSEKQKRDFTITEKQDEVSGQ...
...,...,...,...,...,...,...,...,...
P40004,Sugar transport,Endoplasmic reticulum;Membrane;Transmembrane,Transport,YEA4 YEL004W,UDP-N-acetylglucosamine transporter YEA4,2.A.7.10.5,559292,MWNSLKAFALVFGGCCSNVITFETLMSNETGSINNLITFCQFLFVT...
Q06328,Amino-acid transport,Membrane;Transmembrane,Transport,YPQ2 YDR352W,Probable vacuolar amino acid transporter YPQ2 ...,2.A.43.2.8,559292,MSCSNGIWPTVSNLCGSLSFFTSVISLFPQIIETYRDKSVDGLSPY...
Q12235,Amino-acid transport,Cell membrane;Endoplasmic reticulum;Membrane;T...,Transport,YCT1 YLL055W L0578,High affinity cysteine transporter,2.A.1.14.20,559292,MSKVDVKIGADSISSSDEILVPSRLADVTLAFMEENDAAVPEITPE...


## Clustering

In [4]:
cd_hit(
    executable_location="cd-hit",
    input_fasta=f"../data/datasets/{ORGANISM}_amino_sugar.fasta",
    output_fasta=f"../data/datasets/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}.fasta",
    log_file=LOG_FILE,
    identity_threshold=IDENTITY_THRESHOLD,
    n_threads=N_THREADS,
    memory=4096,
    verbose=True,
)

## Annotations

In [5]:
df_annotations = pd.read_table(f"../data/datasets/{ORGANISM}_amino_sugar.tsv", index_col=0)
df_annotations.head()

,keywords_transport,keywords_location,keywords_transport_related,gene_names,protein_names,tcdb_id,organism_id,sequence
Uniprot,,,,,,,,
P0CD99,Sugar transport,Cell membrane;Membrane;Transmembrane,Transport,MPH2 YDL247W,Alpha-glucosides permease MPH2 (Maltose transp...,NaN,559292,MKNLSFLINRRKENTSDSNVYPGKAKSHEPSWIEMDDQTKKDGLDI...
Q08986,Amino-acid transport,Endoplasmic reticulum;Membrane;Transmembrane,Transport,SAM3 YPL274W,S-adenosylmethionine permease SAM3 (S-adenosyl...,2.A.3.10.15,559292,MDILKRGNESDKFTKIETESTTIPNDSDRSGSLIRRMKDSFKQSNL...
P38967,Amino-acid transport,Membrane;Transmembrane,Transport,TAT2 LTG3 SAB2 SCM2 TAP2 YOL020W,Tryptophan permease (Tryptophan amino acid tra...,2.A.3.10.8,559292,MTEDFISSVKRSNEELKERKSNFGFVEYKSKQLTSSSSHNSNSSHH...
P39003,Sugar transport,Membrane;Transmembrane,Transport,HXT6 YDR343C D9651.12,High-affinity hexose transporter HXT6,NaN,559292,MSQDAAIAEQTPVEHLSAVDSASHSVLSTPSNKAERDEIKAYGEGE...
P38085,Amino-acid transport,Membrane;Transmembrane,Transport,TAT1 TAP1 VAP1 YBR069C YBR0710,Valine/tyrosine/tryptophan amino-acid permease...,2.A.3.10.9,559292,MDDSVSFIAKEASPAQYSHSLHERTHSEKQKRDFTITEKQDEVSGQ...


# Feature generation

## Labels

In [6]:
fasta_to_labels(
    input_fasta=f"../data/datasets/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}.fasta",
    output_tsv=f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_labels.tsv",
)
pd.read_table(
    f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_labels.tsv",
    index_col=0,
).labels.value_counts()

Amino-acid transport    34
Sugar transport         17
Name: labels, dtype: int64

## PSSM

In [7]:
for uniref_cluster_threshold in [50, 90]:
    for psiblast_iterations in [1, 3]:
        calculate_pssm_feature(
            input_fasta=f"../data/datasets/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}.fasta",
            output_tsv="../data/features/{}_amino_sugar_cluster{}_pssm_ur{}_{}it.tsv".format(
                ORGANISM,
                IDENTITY_THRESHOLD,
                uniref_cluster_threshold,
                psiblast_iterations,
            ),
            tmp_folder="../data/intermediate/blast/pssm_uniref{}_{}it".format(
                uniref_cluster_threshold, psiblast_iterations
            ),
            blast_db="../data/raw/uniref/uniref{}/uniref{}.fasta".format(
                uniref_cluster_threshold, uniref_cluster_threshold
            ),
            iterations=psiblast_iterations,
            psiblast_executable="psiblast",
            psiblast_threads=N_THREADS,
            verbose=False,
        )


## Reading dataframes

In [8]:
df_labels = pd.read_table(
    f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_labels.tsv",
    index_col=0,
)
df_pssm_50_1it = pd.read_table(
    f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_pssm_ur50_1it.tsv",
    index_col=0,
)
df_pssm_50_3it = pd.read_table(
    f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_pssm_ur50_3it.tsv",
    index_col=0,
)
df_pssm_90_1it = pd.read_table(
    f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_pssm_ur90_1it.tsv",
    index_col=0,
)
df_pssm_90_3it = pd.read_table(
    f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_pssm_ur90_3it.tsv",
    index_col=0,
)

## Combining dataframes

In [9]:
df_pssm_all = pd.concat(
    [
        df_pssm_50_1it.rename(columns=lambda c: c + "_50_1"),
        df_pssm_50_3it.rename(columns=lambda c: c + "_50_3"),
        df_pssm_90_1it.rename(columns=lambda c: c + "_90_1"),
        df_pssm_90_3it.rename(columns=lambda c: c + "_90_3"),
    ],
    axis=1,
)


## Custom Transformer to try all parameters

In [10]:
class PSSMSelector(BaseEstimator, TransformerMixin):
    def __init__(self, feature_names, uniref_threshold="all", iterations="all"):
        self.feature_names = feature_names
        self.uniref_threshold = uniref_threshold
        self.iterations = iterations

    def fit(self, X, y=None):
        if self.uniref_threshold in {50, 90}:
            has_uniref = (
                np.char.find(self.feature_names, str(self.uniref_threshold)) >= 0
            )
        elif self.uniref_threshold == "all":
            has_uniref = np.array([True] * len(self.feature_names))
        else:
            raise ValueError(f"Incorrect uniref threshold {self.uniref_threshold}")

        if self.iterations in {1, 3}:
            has_iterations = np.char.find(self.feature_names, str(self.iterations)) >= 0
        elif self.iterations == "all":
            has_iterations = np.array([True] * len(self.feature_names))
        else:
            raise ValueError(f"Incorrect iteration count: {self.iterations}")
        self.mask = np.bitwise_and(has_uniref, has_iterations)
        return self

    def transform(self, X, y=None):
        X = np.array(X)
        X = X[:, self.mask]
        return X


# Functions

In [11]:
def get_feature_stats(df_features, df_labels_, labels=["Amino-acid transport", "Sugar transport"]):
    df_stats = pd.concat(
        {
            "corr": df_features.corrwith(
                df_labels_.labels.transform(lambda x: 1.0 if x == labels[1] else 0.0)
            ),
            "mean": df_features.mean(),
            "std": df_features.std(),
        },
        axis=1,
    )

    df_stats["sum"] = df_stats.sum(axis=1)
    df_stats["corr_abs"] = df_stats["corr"].abs()

    df_stats["mean0"] = df_features.loc[df_labels_[df_labels_.labels == labels[0]].index].mean()
    df_stats["mean1"] = df_features.loc[df_labels_[df_labels_.labels == labels[1]].index].mean()

    df_stats["median0"] = df_features.loc[
        df_labels_[df_labels_.labels == labels[0]].index
    ].median()
    df_stats["median1"] = df_features.loc[
        df_labels_[df_labels_.labels == labels[1]].index
    ].median()

    df_stats["mediandiff"] = (df_stats["median0"] - df_stats["median1"]).abs()
    df_stats = df_stats.sort_values("mediandiff", ascending=False)
    return df_stats

In [12]:
def get_independent_test_set(
    df_features, df_labels_, labels=["Amino-acid transport", "Sugar transport"], test_size=0.2
):
    X = df_features.to_numpy()
    y = np.where(df_labels_.labels == labels[1], 1, 0)
    feature_names = df_features.columns.to_numpy(dtype=str)
    sample_names = df_features.index.to_numpy(dtype=str)
    (
        X_train,
        X_test,
        y_train,
        y_test,
        sample_names_train,
        sample_names_test,
    ) = train_test_split(
        X, y, sample_names, stratify=y, random_state=42, shuffle=True, test_size=test_size
    )
    return (
        X_train,
        X_test,
        y_train,
        y_test,
        sample_names_train,
        sample_names_test,
        feature_names,
    )


In [13]:
def print_validation_results(y_true_, y_pred_, labels = ["Amino", "Sugar"]):
    report_dict = classification_report(y_true=y_true_, y_pred=y_pred_, output_dict=True)
    report_dict = {
        labels[0]: report_dict['0'],
        labels[1]: report_dict['1'],
        "Macro": report_dict["macro avg"],
        "Weighted": report_dict["weighted avg"]
    }
    report_df = pd.DataFrame.from_dict(report_dict)
    confusion_matrix_df = pd.DataFrame(
        confusion_matrix(y_true_, y_pred_),
        columns=labels,
        index=labels,
    )
    return report_df, confusion_matrix_df

# Individual Features

## PSSM

### Stats, Plots

In [14]:
df_stats = get_feature_stats(df_pssm_50_1it, df_labels)


df_stats["shapiro_p"] = df_pssm_50_1it.apply(lambda col: shapiro(col)[1], axis=0).round(4)
df_stats["shapiro"] = df_pssm_50_1it.apply(lambda col: shapiro(col)[0], axis=0)

display(df_stats[df_stats.shapiro < 0.9])

df_stats.sort_values("std")

,corr,mean,std,sum,corr_abs,mean0,mean1,median0,median1,mediandiff,shapiro_p,shapiro
CF,-0.197374,0.932608,0.067720,0.802953,0.197374,0.941966,0.913891,0.977004,0.921610,0.055394,0.0001,0.872282
QG,-0.084097,0.951332,0.061902,0.929137,0.084097,0.954976,0.944042,0.969234,1.000000,0.030766,0.0000,0.794973
IN,0.415538,0.116044,0.039127,0.570709,0.415538,0.104661,0.138811,0.101793,0.125000,0.023207,0.0001,0.874040
IP,-0.016834,0.139409,0.053475,0.176049,0.016834,0.140039,0.138148,0.131440,0.139535,0.008095,0.0000,0.863592
ID,0.348440,0.015961,0.035471,0.399871,0.348440,0.007307,0.033267,0.000000,0.000000,0.000000,0.0000,0.522807


,corr,mean,std,sum,corr_abs,mean0,mean1,median0,median1,mediandiff,shapiro_p,shapiro
IR,0.376950,0.144041,0.032514,0.553506,0.376950,0.135460,0.161203,0.133203,0.164336,0.031133,0.2032,0.969117
ID,0.348440,0.015961,0.035471,0.399871,0.348440,0.007307,0.033267,0.000000,0.000000,0.000000,0.0000,0.522807
IK,0.416962,0.119307,0.035603,0.571872,0.416962,0.108913,0.140094,0.106414,0.143411,0.036997,0.0679,0.957904
IE,0.585433,0.079730,0.037387,0.702550,0.585433,0.064405,0.110379,0.061132,0.105085,0.043953,0.0007,0.906805
IS,0.120294,0.268603,0.037610,0.426507,0.120294,0.265435,0.274938,0.267217,0.265306,0.001910,0.8658,0.987534
...,...,...,...,...,...,...,...,...,...,...,...,...
QW,-0.280323,0.255252,0.107725,0.082654,0.280323,0.276394,0.212966,0.286084,0.186235,0.099849,0.0386,0.952110
DC,-0.283971,0.328412,0.107809,0.152249,0.283971,0.349846,0.285543,0.343049,0.229126,0.113923,0.1226,0.963923
QH,-0.277463,0.282291,0.108592,0.113419,0.277463,0.303386,0.240100,0.288323,0.202429,0.085894,0.2660,0.971947
QY,-0.246513,0.235376,0.108813,0.097676,0.246513,0.254157,0.197815,0.237751,0.164609,0.073142,0.0800,0.959577


### Independent test set

In [15]:
(
    X_train,
    X_test,
    y_train,
    y_test,
    sample_names_train,
    sample_names_test,
    feature_names,
) = get_independent_test_set(df_pssm_all, df_labels, test_size=0.2)

### Model selection

SVC (with default RBF kernel) looks the most promising.

In [16]:
for estimator in [
    LinearSVC(max_iter=1e6, class_weight="balanced", random_state=0),
    SVC(class_weight="balanced"),
    RandomForestClassifier(class_weight="balanced", random_state=0),
    LinearSVC(max_iter=1e6, random_state=0),
    SVC(),
    RandomForestClassifier(random_state=0),
    GaussianNB(),
    KNeighborsClassifier(),
    SGDClassifier(random_state=0),
]:
    pipe = make_pipeline(StandardScaler(), estimator)
    scores = cross_val_score(pipe, X_train, y_train, scoring="f1_macro")
    print("### ", str(estimator))
    print(f"CV folds: {scores.round(3)}")
    print(f"Mean: {scores.mean().round(3)}")
    print(f"Std: {scores.std().round(3)}")


###  LinearSVC(class_weight='balanced', max_iter=1000000.0)
CV folds: [1.    0.855 0.873 0.667 0.619]
Mean: 0.803
Std: 0.141
###  SVC(class_weight='balanced')
CV folds: [0.855 0.855 0.855 0.385 0.855]
Mean: 0.761
Std: 0.188
###  RandomForestClassifier(class_weight='balanced')
CV folds: [1.    1.    0.855 0.385 0.667]
Mean: 0.781
Std: 0.233
###  LinearSVC(max_iter=1000000.0)
CV folds: [1.    0.855 0.873 0.667 0.619]
Mean: 0.803
Std: 0.141
###  SVC()
CV folds: [1.    1.    0.667 0.385 0.855]
Mean: 0.781
Std: 0.233
###  RandomForestClassifier()
CV folds: [0.855 1.    0.667 0.385 0.667]
Mean: 0.714
Std: 0.207
###  GaussianNB()
CV folds: [0.5   0.855 0.619 0.385 0.619]
Mean: 0.595
Std: 0.156
###  KNeighborsClassifier()
CV folds: [1.    0.564 0.855 0.385 0.855]
Mean: 0.731
Std: 0.224
###  SGDClassifier()
CV folds: [0.733 0.855 1.    0.564 0.619]
Mean: 0.754
Std: 0.158


### Parameter tuning

In [17]:
gsearch = GridSearchCV(
    estimator=make_pipeline(
        PSSMSelector(feature_names=feature_names),
        StandardScaler(),
        LinearSVC(max_iter=1e6, random_state=0),
    ),
    param_grid={
        "pssmselector__uniref_threshold": [50, 90, "all"],
        "pssmselector__iterations": [1, 3, "all"],
        "linearsvc__class_weight": ["balanced", None],
        "linearsvc__C": [1, 10, 0.1],
        "linearsvc__dual": [True, False],
    },
    cv=5,
    scoring="f1_macro",
    n_jobs=-1,
    return_train_score=True,
)
gsearch.fit(X_train, y_train)
print(gsearch.best_params_)
print(gsearch.best_score_)
best_estimator_svc = gsearch.best_estimator_


{'linearsvc__C': 1, 'linearsvc__class_weight': 'balanced', 'linearsvc__dual': True, 'pssmselector__iterations': 1, 'pssmselector__uniref_threshold': 'all'}
0.8534487734487733


### Dimensionality reduction

In [18]:
pca = PCA()
pca.fit(X_train)
csum = np.cumsum(pca.explained_variance_ratio_)
print("Number of components to explain 97% of variance:", np.argmax(csum >= 0.97) + 1)

Number of components to explain 97% of variance: 22


In [19]:
gsearch = GridSearchCV(
    estimator=make_pipeline(
        PSSMSelector(feature_names=feature_names),
        StandardScaler(),
        PCA(),
        StandardScaler(),
        LinearSVC(max_iter=1e6, random_state=0),
    ),
    param_grid={
        "pssmselector__uniref_threshold": [50, 90, "all"],
        "pssmselector__iterations": [1, 3, "all"],
        "linearsvc__class_weight": ["balanced", None],
        "linearsvc__C": [1, 10, 100],
        "linearsvc__dual": [True, False],
        "pca__n_components": np.linspace(0.8, 0.99, 20)
    },
    cv=5,
    scoring="f1_macro",
    n_jobs=-1,
    return_train_score=True,
)
gsearch.fit(X_train, y_train)
print(gsearch.best_params_)
print(gsearch.best_score_)
best_estimator_lsvc_pca = gsearch.best_estimator_

{'linearsvc__C': 10, 'linearsvc__class_weight': 'balanced', 'linearsvc__dual': True, 'pca__n_components': 0.86, 'pssmselector__iterations': 1, 'pssmselector__uniref_threshold': 50}
0.8788455988455988


In [20]:
gsearch = GridSearchCV(
    estimator=make_pipeline(
        PSSMSelector(feature_names=feature_names),
        StandardScaler(),
        PCA(),
        StandardScaler(),
        SVC(),
    ),
    param_grid={
        "pssmselector__uniref_threshold": [50, 90, "all"],
        "pssmselector__iterations": [1, 3, "all"],
        "svc__class_weight": ["balanced", None],
        "svc__C": [1, 10, 100],
        # "svc__gamma": ["scale", 1, 0.1, 0.01],
        "pca__n_components": np.linspace(0.8, 0.99, 20)
    },
    cv=5,
    scoring="f1_macro",
    n_jobs=-1,
    return_train_score=True,
)
gsearch.fit(X_train, y_train)
print(gsearch.best_params_)
print(gsearch.best_score_)
best_estimator_svc_pca = gsearch.best_estimator_

{'pca__n_components': 0.99, 'pssmselector__iterations': 3, 'pssmselector__uniref_threshold': 50, 'svc__C': 1, 'svc__class_weight': 'balanced'}
0.9333333333333332


### Validation

In [21]:
best_estimator = best_estimator_svc_pca
best_scores = cross_val_score(
    estimator=clone(best_estimator), X=X_train, y=y_train, scoring="f1_macro"
)
print(f"Train scores: {best_scores.mean().round(3)}+-{best_scores.std().round(3)}")

y_pred = best_estimator.predict(X_test)
y_true = y_test.copy()

report_df, confusion_matrix_df = print_validation_results(y_true, y_pred, labels=["Amino", "Sugar"])
display(report_df.round(3))
display(confusion_matrix_df)

Train scores: 0.933+-0.133


,Amino,Sugar,Macro,Weighted
precision,0.700,1.00,0.850,0.809
recall,1.000,0.25,0.625,0.727
f1-score,0.824,0.40,0.612,0.670
support,7.000,4.00,11.000,11.000


,Amino,Sugar
Amino,7,0
Sugar,3,1


In [22]:
best_estimator = best_estimator_lsvc_pca
best_scores = cross_val_score(
    estimator=clone(best_estimator), X=X_train, y=y_train, scoring="f1_macro"
)
print(f"Train scores: {best_scores.mean().round(3)}+-{best_scores.std().round(3)}")

y_pred = best_estimator.predict(X_test)
y_true = y_test.copy()

report_df, confusion_matrix_df = print_validation_results(y_true, y_pred, labels=["Amino", "Sugar"])
display(report_df.round(3))
display(confusion_matrix_df)

Train scores: 0.879+-0.122


,Amino,Sugar,Macro,Weighted
precision,0.750,0.667,0.708,0.720
recall,0.857,0.500,0.679,0.727
f1-score,0.800,0.571,0.686,0.717
support,7.000,4.000,11.000,11.000


,Amino,Sugar
Amino,6,1
Sugar,2,2


# Alternative Eval

In [23]:
from sklearn.model_selection import LeaveOneOut

X = df_pssm_all.values
labels = df_labels.labels
y = np.where(labels == "Sugar transport", 1, 0)


params_svc = {
    "svc__class_weight": ["balanced"],
    "svc__C": [0.1, 1, 10],
    # "svc__gamma": ["scale", 1e-1, 1e-2, 1e-3],
    "pssmselector__uniref_threshold": [50, 90, "all"],
    "pssmselector__iterations": [1, 3, "all"],
}
gsearch = GridSearchCV(
    make_pipeline(PSSMSelector(feature_names=feature_names), StandardScaler(), SVC()),
    param_grid=params_svc,
    cv=LeaveOneOut(),
    scoring="f1_macro",
    return_train_score=True,
    refit=True,
)
# the score does not make a difference, since it's either 1 (correct) or 0 (incorrect)
res = cross_val_predict(gsearch, X, y, n_jobs=-1, cv=LeaveOneOut())
print(f1_score(y, res, average="macro"))


params_svc = {
    "svc__class_weight": ["balanced"],
    "svc__C": [0.1, 1, 10],
    # "svc__gamma": ["scale", 1e-1, 1e-2, 1e-3],
    "pca__n_components": [0.8, 0.9],
    "pssmselector__uniref_threshold": [50, 90, "all"],
    "pssmselector__iterations": [1, 3, "all"],
}
gsearch = GridSearchCV(
    make_pipeline(
        PSSMSelector(feature_names=feature_names),
        StandardScaler(),
        PCA(),
        StandardScaler(),
        SVC(),
    ),
    param_grid=params_svc,
    cv=LeaveOneOut(),
    scoring="f1_macro",
    return_train_score=True,
    refit=True,
)
# the score does not make a difference, since it's either 1 (correct) or 0 (incorrect)
res = cross_val_predict(gsearch, X, y, n_jobs=-1, cv=LeaveOneOut())
print(f1_score(y, res, average="macro"))


0.8432147562582346
0.8583333333333334
